In [ ]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3

In [ ]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [ ]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

In [ ]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    print("Page : " + url)
    response = requests.get(url)
    content_source = response.text
    return html.document_fromstring(content_source)

In [ ]:
def get_row(doc, chef, total):
    nom = doc.xpath('//h1[@itemprop="name"]/text()')[0]
    photo = doc.xpath('//img[@class="chef-picture"]/@src')[0]
    if doc.xpath('//div[contains(@class,"chef-badge")]/span[@class="top"]/text()'):
        valide = 'OUI'
    else:
        valide = 'NON'
    texte_intro = doc.xpath('//h3[@class="chef-tagline"]/text()')[0]
    note = doc.xpath('//h4[contains(@itemprop,"ratingValue")]/@content')[0]
    avis_nb = doc.xpath('//meta[contains(@itemprop,"ratingCount")]/@content')[0]
    a_propos = doc.xpath('//div[contains(@class,"chef-card")]//span[@class="full"]/text()')[0]
    zone_ville = doc.xpath('//span[contains(@itemprop,"addressLocality")]/text()')[0]
    zone_pays_code = doc.xpath('//meta[contains(@itemprop,"addressCountry")]/@content')[0]
    if doc.xpath('//div[contains(@class,"chef-card")]/p[2]/text()'):
        langues = doc.xpath('//div[contains(@class,"chef-card")]/p[2]/text()')[0]
    else:
        langues = 'NA'
    cuisines_types = doc.xpath('//span[@itemprop="servesCuisine"]/text()') # list
    questions = doc.xpath('//div[@class="chef-question"]/p/text()')
    row = [chef, total, nom, photo, valide, texte_intro, note, avis_nb, 
           a_propos, zone_ville, zone_pays_code, langues, cuisines_types, questions]
    return row

In [ ]:
columns = ['chef', 'total', 'nom', 'photo', 'valide', 'texte_intro', 'note', 'avis_nb', 
           'a_propos', 'zone_ville', 'zone_pays_code','langues', 'cuisines_types', 'questions']
df_output = pd.DataFrame(columns=columns)

In [ ]:
for index, row in df_source.iterrows():
    doc = getChefPage(row['chef'])
    df_output.loc[index] = get_row(doc, row['chef'], row['total'])

In [ ]:
df_output.head()

In [ ]:
df_output.info()

In [ ]:
# database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
# table = 'chefs'
# conn = sqlite3.connect(database_file)
# df_output.to_sql(table, conn, if_exists="replace")

In [ ]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_details.xlsx'.format(date_jour), index=False, header=True)